# QM9 Pro.
Not sure what I’m supposed to call this file.  
Plan is to keep this repo much neater than before.  
Getting to work now.

In [68]:
import torch_geometric
import torch
from torch.nn import ModuleList, Module, Sequential, Embedding, Linear, BatchNorm1d, GELU, MSELoss
from torch.optim import Adam
import torch.optim.lr_scheduler as lr_scheduler
from typing import Any
from torch_geometric.nn import MessagePassing
from torch_geometric.nn.pool import global_mean_pool
from torch_geometric.datasets import QM9
from torch_geometric.data import DataLoader
import wandb

In [47]:
wandb.login()
torch.manual_seed(2002)

In [102]:
dataset = QM9(root='QM9/')

/usr/local/Caskroom/miniconda/base/envs/GDL/lib/python3.11/site-packages/torch_geometric/data/dataset.py:242: UserWarning: The `pre_transform` argument differs from the one used in the pre-processed version of this dataset. If you want to make use of another pre-processing technique, pass `force_reload=True` explicitly to reload the dataset.
  warnings.warn(


In [49]:
config_1 = {
    "base_learning_rate": 1e-3,
    "architecture": "Sparse 2-layer MPNN",
    "optimizer": "Adam",
    "scheduler": "ReduceLROnPlateau",
    "dataset": "QM9",
    "epochs": 10,
    "batch_size": 128,
}

config_2 = {
    "base_learning_rate": 1e-3,
    "architecture": "Sparse 2-layer MPNN",
    "optimizer": "Adam",
    "scheduler": "COSINEANNEALINGLR",
    "dataset": "QM9",
    "epochs": 10,
    "batch_size": 128,
}

In [138]:
class MPNNLayer(MessagePassing):
    def __init__(self):
        super().__init__(aggr='add')
        
        self.emb_dim = 32
        
        self.mlp_msg = Sequential(
            Linear(2*self.emb_dim + 1, self.emb_dim), BatchNorm1d(self.emb_dim), GELU()
          )
        
        self.mlp_upd = Sequential(
            Linear(2*self.emb_dim, self.emb_dim), BatchNorm1d(self.emb_dim), GELU()
          )

    def forward(self, h, edge_index, edge_attr):
        out = self.propagate(edge_index, h=h, edge_attr=edge_attr)
        return out

    def message(self, h_i, h_j, edge_attr):
        msg = torch.cat((h_i, h_j, edge_attr), dim=-1)
        return self.mlp_msg(msg)
    
    def aggregate(self, inputs, index):
        return scatter_add_(inputs, index, dim=self.node_dim)
    
    def update(self, aggr_out, h):
        upd_out = torch.cat([h, aggr_out], dim=-1)
        return self.mlp_upd(upd_out)

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}(emb_dim={self.emb_dim}, aggr={self.aggr})')

In [139]:
class MPNNModel(Module):
    def __init__(self):
        super().__init__()
        self.emb_dim = 32
        self.num_mp_layers = 2
        
        self.embedding = Embedding(118, self.emb_dim)
        
        self.convs = torch.nn.ModuleList()
        for layer in range(self.num_mp_layers):
            self.convs.append(MPNNLayer())
        
        self.pool = global_mean_pool

        self.lin_pred = Sequential(
            Linear(self.emb_dim, 8), GELU(),
            Linear(self.emb_dim, 1), GELU()
        )
        
    def forward(self, data):
        h = self.embedding(data.x)
        
        for conv in self.convs:
            h = h + conv(h, data.edge_index, data.edge_attr)

        h_graph = self.pool(h, data.batch)

        out = self.lin_pred(h_graph)

        return out.view(-1)

In [140]:
model = MPNNModel()

In [141]:
dataloader_1000 = DataLoader(dataset[:1000], batch_size=128, shuffle=True)

/usr/local/Caskroom/miniconda/base/envs/GDL/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [142]:
loss_fn = MSELoss()
optimizer = Adam(model.parameters())

for data in dataloader_1000:
    optimizer.zero_grad()
    y_pred = model(data)
    loss = loss_fn(ypred, data.y)
    optimizer.step()

RuntimeError: Tensors must have same number of dimensions: got 2 and 1

In [ ]:
wandb.init(
    project="QM9-Pro",
    config = config_1
)

# train

wandb.finish()